## Capstone 1
To import the dataset I have simply copied the table from wikipedia into excel and save it as a csv file

In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
#%matplotlib notebook
%matplotlib inline
import folium
from geopy.geocoders import Nominatim
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Libraries Imported")

Libraries Imported


## Read and check the dataset

In [3]:
df1=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Input_Table_01.csv")
print(df1.head())
print(df1.shape)
print(df1.columns)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(288, 3)
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


### Eliminating the rows that don't have an assigned Borough and assigning the name of the Borough to those remaining rows with a not assigned Neighbourhood

In [4]:
print('Before Splitting',np.unique(df1['Borough'].values))
df2=df1[df1['Borough']!='Not assigned']
print('Before Splitting',np.unique(df2['Borough'].values))
print('size of df2: ',df2.shape)
positions=[]
for i in range(len(df2)):
    if df2.iloc[i,2]=='Not assigned':
        positions.append(i)
print('Before Modification',df2.iloc[positions])
for i in range(len(df2)):
    if df2.iloc[i,2]=='Not assigned':
        df2.iloc[i,2]=df2.iloc[i,1]
print('After Modification',df2.iloc[positions])
df2=df2.reset_index(drop=True)
print(df2.iloc[28,2])
df2.iloc[28,2]='Cedarvale'
df2.iloc[38,2]='Caledonia'
df2.iloc[70,2]='Downsview Airport'
df2.iloc[102,2]='Marina Del Ray'
df2.iloc[136,2]='Canada Post Gateway'
df2.iloc[136,2]='Humber Bay'
df2=df2.drop(169)
df2=df2.reset_index(drop=True)
df2.iloc[170,2]='Humber Bay'
df2.iloc[174,2]='Beaumonde Heights'
df2.iloc[183,2]='Toronto Station A'
df2.iloc[196,2]='969 Eastern'
df2.iloc[209,2]='Bloor St'

print(df2.head())


Before Splitting ['Central Toronto' 'Downtown Toronto' 'East Toronto' 'East York'
 'Etobicoke' 'Mississauga' 'North York' 'Not assigned' "Queen's Park"
 'Scarborough' 'West Toronto' 'York']
Before Splitting ['Central Toronto' 'Downtown Toronto' 'East Toronto' 'East York'
 'Etobicoke' 'Mississauga' 'North York' "Queen's Park" 'Scarborough'
 'West Toronto' 'York']
size of df2:  (211, 3)
Before Modification   Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned
After Modification   Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Queen's Park
Humewood-Cedarvale
  Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights


C:\Users\Piero\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Piero\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [4]:
gropus_by_Postcode=df2.groupby(by='Postcode')
postcode=[]
borough=[]
neig=[]
for name,value in gropus_by_Postcode:
    postcode.append(value['Postcode'].iloc[0])
    borough.append(value['Borough'].iloc[0])
    neig.append(', '.join(value['Neighbourhood']))
df3=pd.DataFrame({'Postcode':postcode,'Borough':borough,'Neighbourhood':neig})
print(df3.head())
print(df3.shape)
print(df3.columns)

  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae
(103, 3)
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


## Loading the dataset with the geographical coordinates

In [6]:
latitude=[]
longitude=[]
for i in range(len(df2)):
    address = '{}, Toronto, ON'.format(df2['Neighbourhood'].iloc[i])
    geolocator = Nominatim(user_agent="to_explorer")
    location = geolocator.geocode(address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print(i)

    #print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
print(latitude)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
[43.7587999, 43.732658, 43.6400801, 43.6607056, 43.7227784, 43.7220788, 43.6599803, 43.6393743, 43.8049304, 43.8091955, 43.737178, 43.7120785, 43.7062977, 43.65815275, 43.6565019, 43.7087117, 43.633637, 43.6453376, 43.653963, 43.6404664, 43.6631995, 43.7901172, 43.78027

In [16]:

# df2['Latitude']=latitude
# df2['Longitude']=longitude

# df_toronto=df2.copy()

# print(df_toronto.shape)
# df_toronto

#df_toronto.to_csv("C:\\Users\\Piero\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Lat_Long.csv")

df_toronto=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Lat_Long.csv",index_col=0)
df_toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
4,M6A,North York,Lawrence Heights,43.722778,-79.450933
5,M6A,North York,Lawrence Manor,43.722079,-79.437507
6,M7A,Queen's Park,Queen's Park,43.659980,-79.390369
7,M9A,Etobicoke,Islington Avenue,43.639374,-79.521218
8,M1B,Scarborough,Rouge,43.804930,-79.165837
9,M1B,Scarborough,Malvern,43.809196,-79.221701


In [17]:
latitude=43.6532
longitude=-79.3832
# create map of New York using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Loading the New York Dataset

In [18]:
df_New_York=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\New_York_Projects_neighbor.csv")
del df_New_York['Unnamed: 0']
df_New_York.head()


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [19]:
latitude=40.7128
longitude=-74.0060
# create map of New York using latitude and longitude values

map_new_york = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_New_York['Latitude'], df_New_York['Longitude'], df_New_York['Borough'], df_New_York['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_new_york)  
    
map_new_york

## Selecting only the boroughs that contain the word Toronto.

In [20]:
df_tor_cut=df_toronto[df_toronto['Borough'].str.contains('Toronto')]
print(df_tor_cut.shape)
print(df_tor_cut.head())
print(df_tor_cut['Borough'].value_counts())

(73, 5)
   Postcode           Borough    Neighbourhood   Latitude  Longitude
2       M5A  Downtown Toronto     Harbourfront  43.640080 -79.380150
3       M5A  Downtown Toronto      Regent Park  43.660706 -79.360457
13      M5B  Downtown Toronto          Ryerson  43.658153 -79.379080
14      M5B  Downtown Toronto  Garden District  43.656502 -79.377128
27      M5C  Downtown Toronto   St. James Town  43.669403 -79.372704
Downtown Toronto    36
Central Toronto     17
West Toronto        13
East Toronto         7
Name: Borough, dtype: int64


## Starting Analysis

In [1]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '...' # Foursquare API version



## Defining the function to analyse different neighborhoods.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=800,LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
tor_venues = getNearbyVenues(names=df_toronto['Neighbourhood'], latitudes=df_toronto['Latitude'], longitudes=df_toronto['Longitude'])

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
Downsview Airport
Downsview East
The Danforth West
Riverdale
Design Exchange
Toron

In [25]:
print(tor_venues.shape)
print(tor_venues.head())
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

tor_venues.to_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Venues_800m.csv")

(9251, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Parkwoods                43.7588              -79.320197   
1    Parkwoods                43.7588              -79.320197   
2    Parkwoods                43.7588              -79.320197   
3    Parkwoods                43.7588              -79.320197   
4    Parkwoods                43.7588              -79.320197   

             Venue  Venue Latitude  Venue Longitude        Venue Category  
0  Allwyn's Bakery       43.759840       -79.324719  Caribbean Restaurant  
1             LCBO       43.757774       -79.314257          Liquor Store  
2      Tim Hortons       43.760668       -79.326368                  Café  
3       A&W Canada       43.760643       -79.326865  Fast Food Restaurant  
4        Dollarama       43.757317       -79.312578        Discount Store  
There are 353 uniques categories.


In [25]:
new_york_venues = getNearbyVenues(names=df_New_York['Neighborhood'], latitudes=df_New_York['Latitude'], longitudes=df_New_York['Longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [26]:
new_york_venues.to_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\New_York_Venues_800m.csv")

## Analysis of the venues

In [ ]:
# Loading the two files we have created



In [23]:
 print(tor_venues['Neighborhood'].head())
 np.unique(tor_venues['Neighborhood'].values)

0    The Beaches
1    The Beaches
2    The Beaches
3    The Beaches
4    The Beaches
Name: Neighborhood, dtype: object


array(['Adelaide, King, Richmond', 'Berczy Park',
       'Brockton, Exhibition Place, Parkdale Village',
       'Business Reply Mail Processing Centre 969 Eastern',
       'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
       'Cabbagetown, St. James Town', 'Central Bay Street',
       'Chinatown, Grange Park, Kensington Market', 'Christie',
       'Church and Wellesley', 'Commerce Court, Victoria Hotel',
       'Davisville', 'Davisville North',
       'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West',
       'Design Exchange, Toronto Dominion Centre',
       'Dovercourt Village, Dufferin',
       'First Canadian Place, Underground city',
       'Forest Hill North, Forest Hill West',
       'Harbord, University of Toronto',
       'Harbourfront East, Toronto Islands, Union Station',
       'Harbourfront, Regent Park', 'High Park, The Junction South',
       'Lawrence Park', 'Little Portugal, Trinity',
     

In [19]:
# one hot encoding
tor_venues_hot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
print(tor_venues_hot.shape)
# add neighborhood column back to dataframe
tor_venues_hot['Neighbourhood'] = tor_venues['Neighborhood']
print(tor_venues_hot.shape)
# move neighborhood column to the first column
fixed_columns = [tor_venues_hot.columns[-1]] + list(tor_venues_hot.columns[:-1])
tor_venues_hot = tor_venues_hot[fixed_columns]

print(tor_venues_hot.shape)
tor_venues_hot.head()

(1705, 231)
(1705, 232)
(1705, 232)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
tor_venues_group = tor_venues_hot.groupby('Neighbourhood').mean().reset_index()
print(tor_venues_group.shape)
tor_venues_group.head()

(38, 232)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.03,0.0,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.00,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


## Finding the top venues for each  Neighbourhood

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_venues_group['Neighbourhood']

for ind in np.arange(tor_venues_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_venues_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Café,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Gym,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Garden Center,Garden
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Plane,Bar


## Clustering Analysis

In [29]:
# set number of clusters
kclusters = 5

tor_venues_clustering = tor_venues_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_tor_cut_merge = df_tor_cut.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_tor_cut_merge = df_tor_cut_merge.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_tor_cut_merge.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Sandwich Place
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run


In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_cut_merge['Latitude'], df_tor_cut_merge['Longitude'], df_tor_cut_merge['Neighbourhood'], df_tor_cut_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [35]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 0, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
42,East Toronto,0,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Sandwich Place
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant
45,Central Toronto,0,Breakfast Spot,Hotel,Food & Drink Shop,Clothing Store,Dance Studio
46,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Furniture / Home Store,Park
47,Central Toronto,0,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Pharmacy
49,Central Toronto,0,Coffee Shop,Pub,Fried Chicken Joint,Restaurant,Bagel Shop
51,Downtown Toronto,0,Coffee Shop,Park,Café,Restaurant,Italian Restaurant
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant
53,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Restaurant


In [36]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 1, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,Central Toronto,1,Playground,Park,Restaurant,Yoga Studio,Discount Store
50,Downtown Toronto,1,Park,Playground,Trail,Building,Yoga Studio


In [37]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 2, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run


In [38]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 3, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,Central Toronto,3,Music Venue,Garden,Yoga Studio,Fast Food Restaurant,Farmers Market


In [39]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 4, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,East Toronto,4,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail
